In [15]:
import numpy
from numpy import trace,cov,iscomplexobj
from numpy.random import random,randint,randn
from scipy.linalg import sqrtm

In [6]:
#calculate frechet inspection score
def calc_frechet_score(act1,act2):
    #calc mean and covariance for first activation
    mean1,cov1 = act1.mean(axis=0), cov(act1,rowvar=False)
    mean2,cov2 = act2.mean(axis=0), cov(act2,rowvar=False)
    
    #calc sum of square difference btw means
    ssdiff = numpy.sum((mean1-mean2) ** 2.0)
    
    #calcualte sqrt of product between cov
    covmean = sqrtm(cov1.dot(cov2))
    
    #check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
        
    fid = ssdiff + trace(cov1+cov2 - 2.0 * covmean)
    return fid

In [7]:
# defining 2 random activations
# the output layer for inception model has 2048 activation features
act1 = random(10*2048)
act1 = act1.reshape((10,2048))

act2 = random(10*2048)
act2 = act2.reshape((10,2048))

In [10]:
#fid between act1 and act1
print("Comparing same activations",calc_frechet_score(act1,act1))

#fid between act1 and act2
print("Comparing different activations",calc_frechet_score(act1,act2))

Comparing same activations -0.00015799757581247062
Comparing different activations 353.74969235921543


### Implementing FID with keras


In [11]:
import tensorflow
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from skimage.transform import resize

In [13]:
# when we remove the top output layer of the model it also removes the global
# avg pooling layer that we require. hence we get it back by specifying pooling='avg'
model = InceptionV3(include_top=False,pooling='avg',input_shape=(299,299,3))


87916544/87910968 [==============================] - 156s 2us/step


In [19]:
# frechet inception distance
#calculate frechet inspection score
def calc_frechet_score(model,images1,images2):
    
    act1 = model.predict(images1)
    act2 = model.predict(images2)
    
    #calc mean and covariance for first activation
    mean1,cov1 = act1.mean(axis=0), cov(act1,rowvar=False)
    mean2,cov2 = act2.mean(axis=0), cov(act2,rowvar=False)
    
    #calc sum of square difference btw means
    ssdiff = numpy.sum((mean1-mean2) ** 2.0)
    
    #calcualte sqrt of product between cov
    covmean = sqrtm(cov1.dot(cov2))
    
    #check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
        
    fid = ssdiff + trace(cov1+cov2 - 2.0 * covmean)
    return fid

In [12]:
import numpy as np
# scale the image to a new size as not all the time we will have image of required sizes
def scale_image(images,new_shape):
    image_list = list()
    for img in images:
        new_img = resize(img,new_shape,0)
        image_list.append(new_img)
    return np.array(image_list)


In [16]:
# check on 10 32x32 images with random pixels from [0,255]
images1 = randint(0,255,10*32*32*3)
images1 = images1.reshape((10,32,32,3))

images2 = randint(0,255,10*32*32*3)
images2 = images2.reshape((10,32,32,3))


In [17]:
# convert the integer pixel values to floating values and then scale them
images1 = images1.astype('float32')
images2 = images2.astype('float32')

#resize image
images1 = scale_image(images1,(299,299,3))
images2 = scale_image(images2,(299,299,3))

In [18]:
# preprocess image
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)

In [20]:
#fid between images1 and images2
fid = calc_frechet_score(model,images1,images2)
print(fid)

41.45074737170265


In [21]:
fid = calc_frechet_score(model,images1,images1)
print(fid)

-2.539265177219047e-05
